# Data Analyzer with Dynamic Time Warping

In [1]:
%%time
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import dataframe_image as dfi #Optional for saving df as picture
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

Wall time: 969 ms


In [2]:
dataset_all = pd.read_excel('all_sentiment_and_price_values_preprocessed_final_cogn2022.xlsx')
start_date = '01-01-2011'
end_date = '04-01-2021'
selected_date = (dataset_all['Date'] >= start_date) & (dataset_all['Date'] <= end_date)
dataset = dataset_all.loc[selected_date]
#dataset.head()

# Analysis with Dynamic Time Warping 
(Source: https://github.com/e-alizadeh/medium/blob/master/notebooks/intro_to_dtw.ipynb)

In [3]:
dataset

,Unnamed: 0,Date,Price_change_normalized,Price_normalized,Daily_Sum_RavenPack_normalized,Csum_RavenPack_normalized,Daily_Sum_FinBERT_normalized,Csum_FinBERT_normalized,Daily_Sum_CrudeBERT_normalized,Csum_CrudeBERT_normalized,Daily_Sum_CrudeBERT_GT_normalized,Csum_CrudeBERT_GT_normalized,Daily_Sum_CrudeBERT_Plus_normalized,Csum_CrudeBERT_Plus_normalized,Daily_Sum_CrudeBERT_Plus_GT_normalized,Csum_CrudeBERT_Plus_GT_normalized
0,1,2012-01-01,NaN,NaN,0.290618,-0.881584,-0.076923,0.996476,0.224665,0.214182,0.137787,0.520282,0.040244,0.993878,0.020886,0.997670
1,2,2012-01-02,NaN,NaN,0.409611,-0.872358,-0.230769,0.995154,0.579376,0.241237,0.418438,0.539966,0.115556,0.997416,0.074770,0.999603
2,3,2012-01-03,NaN,0.849383,0.144165,-0.869012,0.384615,0.995595,0.090034,0.245914,0.102908,0.544678,-0.239179,0.987955,-0.233376,0.992445
3,4,2012-01-04,-0.120777,0.854556,-0.034325,-0.869620,0.692308,0.996916,-0.181912,0.238154,-0.165684,0.536643,0.011107,0.987665,0.001614,0.992220
4,5,2012-01-05,-0.246543,0.826502,0.217391,-0.864653,0.846154,0.998678,-0.133834,0.232592,-0.124270,0.530573,0.003259,0.987088,-0.006788,0.991747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3374,3375,2021-03-28,-0.140399,NaN,-0.075515,0.989862,0.230769,-0.994714,-0.129712,0.385501,-0.075579,-0.249496,-0.162733,0.284650,-0.109355,-0.061653
3375,3376,2021-03-29,-0.065207,0.025667,0.450801,1.000000,-0.692308,-0.997357,0.250674,0.397524,0.280133,-0.236374,0.177268,0.290450,0.170357,-0.056900
3376,3377,2021-03-30,-0.267885,0.005571,-0.208238,0.995539,-0.076923,-0.998238,0.088531,0.402132,0.098732,-0.231861,0.373323,0.303434,0.332951,-0.047350
3377,3378,2021-03-31,-0.318776,-0.022085,-0.258581,0.989963,0.230769,-0.998238,-0.243589,0.391551,-0.214747,-0.242224,0.202254,0.310149,0.176660,-0.042411


In [4]:
def compute_euclidean_distance_matrix(x, y) -> np.array:

    """Calculate distance matrix

    This method calculates the pairwise Euclidean distance between two sequences.

    The sequences can have different lengths.

    """

    dist = np.zeros((len(y), len(x)))

    for i in range(len(y)):

        for j in range(len(x)):

            dist[i,j] = (x[j]-y[i])**2

    return dist


def compute_accumulated_cost_matrix(x, y) -> np.array:

    """Compute accumulated cost matrix for warp path using Euclidean distance

    """

    distances = compute_euclidean_distance_matrix(x, y)
    # Initialization

    cost = np.zeros((len(y), len(x)))

    cost[0,0] = distances[0,0]
    for i in range(1, len(y)):

        cost[i, 0] = distances[i, 0] + cost[i-1, 0]  
    for j in range(1, len(x)):

        cost[0, j] = distances[0, j] + cost[0, j-1]  
    # Accumulated warp path cost

    for i in range(1, len(y)):

        for j in range(1, len(x)):

            cost[i, j] = min(

                cost[i-1, j],    # insertion

                cost[i, j-1],    # deletion

                cost[i-1, j-1]   # match

            ) + distances[i, j] 
    return cost


mpl.rcParams['figure.dpi'] = 150

savefig_options = dict(format="png", dpi=150, bbox_inches="tight")


NameError: name 'mpl' is not defined

In [5]:
Price = "Price_normalized"
RavenPack = "Csum_RavenPack_normalized"
FinBERT = "Csum_FinBERT_normalized"
CrudeBERT = "Csum_CrudeBERT_normalized"
CrudeBERT_GT = "Csum_CrudeBERT_GT_normalized"
CrudeBERT_Plus = "Csum_CrudeBERT_Plus_normalized"
CrudeBERT_Plus_GT = "Csum_CrudeBERT_Plus_GT_normalized"

selected_data = ['Date', Price, RavenPack, FinBERT, CrudeBERT, CrudeBERT_GT, CrudeBERT_Plus, CrudeBERT_Plus_GT]


dataset_interpolate = dataset_all[selected_data].select_dtypes(exclude='datetime64[ns]').interpolate(method="linear")
dataset_interpolate = dataset_interpolate.dropna()
dataset_interpolate["Date"] = dataset_all["Date"] #Filling in NaN in a Series via linear interpolation

dataset_propagate = dataset_all[selected_data].fillna(method="ffill")
dataset_propagate = dataset_propagate.dropna() #Propagates NaN-values forward or backward

dataset_drop = dataset_all[selected_data].dropna() #NaN-values are dropped

dataset = dataset_propagate

average_price = dataset[Price].mean()
dataset[Price] = dataset.apply(lambda row: row[Price] - average_price, axis=1)

start_date = '01-01-2012'
end_date = '04-01-2021'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

name_of_df = 'DTW-Distance to Price'
df_dtw = pd.DataFrame(data={name_of_df: ['RavenPack', 'FinBERT', 'CrudeBERT', 'CrudeBERT_GT', 'CrudeBERT_Plus', 'CrudeBERT_Plus_GT'],
                             '01-01-2012': [0,0,0,0,0,0],'01-01-2013': [0,0,0,0,0,0],'01-01-2014': [0,0,0,0,0,0],
                             '01-01-2015': [0,0,0,0,0,0],'01-01-2016': [0,0,0,0,0,0],'01-01-2017': [0,0,0,0,0,0], 
                             '01-01-2018': [0,0,0,0,0,0],'01-01-2019': [0,0,0,0,0,0],'01-01-2020': [0,0,0,0,0,0]})

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2012'
end_date = '01-01-2013'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2013'
end_date = '01-01-2014'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2014'
end_date = '01-01-2015'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2015'
end_date = '01-01-2016'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2016'
end_date = '01-01-2017'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2017'
end_date = '01-01-2018'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2018'
end_date = '01-01-2019'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2019'
end_date = '01-01-2020'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2020'
end_date = '01-01-2021'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

#df_corr = df_corr.set_index('Labels')
df_dtw_picture = df_dtw.style.background_gradient(cmap='RdYlGn_r',axis=0)
#dfi.export(df_dtw_picture,'../results/'+name_of_df+'.png')
df_dtw_picture

,DTW-Distance to Price,01-01-2012,01-01-2013,01-01-2014,01-01-2015,01-01-2016,01-01-2017,01-01-2018,01-01-2019,01-01-2020
0,RavenPack,506,541,442,188,18,162,103,171,360
1,FinBERT,137,70,72,330,270,126,31,118,110
2,CrudeBERT,114,111,32,108,199,311,135,40,133
3,CrudeBERT_GT,34,79,23,79,28,164,54,79,47
4,CrudeBERT_Plus,34,80,79,34,148,273,92,42,41
5,CrudeBERT_Plus_GT,47,59,67,103,57,136,29,46,39


In [6]:
Price = "Price_normalized"
RavenPack = "Csum_RavenPack_normalized"
FinBERT = "Csum_FinBERT_normalized"
CrudeBERT = "Csum_CrudeBERT_normalized"
CrudeBERT_GT = "Csum_CrudeBERT_GT_normalized"
CrudeBERT_Plus = "Csum_CrudeBERT_Plus_normalized"
CrudeBERT_Plus_GT = "Csum_CrudeBERT_Plus_GT_normalized"

selected_data = ['Date', Price, RavenPack, FinBERT, CrudeBERT, CrudeBERT_GT, CrudeBERT_Plus, CrudeBERT_Plus_GT]


dataset_interpolate = dataset_all[selected_data].select_dtypes(exclude='datetime64[ns]').interpolate(method="linear")
dataset_interpolate = dataset_interpolate.dropna()
dataset_interpolate["Date"] = dataset_all["Date"] #Filling in NaN in a Series via linear interpolation

dataset_propagate = dataset_all[selected_data].fillna(method="ffill")
dataset_propagate = dataset_propagate.dropna() #Propagates NaN-values forward or backward

dataset_drop = dataset_all[selected_data].dropna() #NaN-values are dropped

dataset = dataset_propagate

average_price = dataset[Price].mean()
dataset[Price] = dataset.apply(lambda row: row[Price] - average_price, axis=1)

start_date = '01-01-2012'
end_date = '04-01-2021'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

name_of_df = 'DTW-Distance to Price'
df_dtw = pd.DataFrame(data={name_of_df: ['RavenPack', 'FinBERT', 'CrudeBERT', 'CrudeBERT_GT', 'CrudeBERT_Plus', 'CrudeBERT_Plus_GT'],
                             '01-01-2012': [0,0,0,0,0,0],'01-01-2013': [0,0,0,0,0,0],'01-01-2014': [0,0,0,0,0,0],
                             '01-01-2015': [0,0,0,0,0,0],'01-01-2016': [0,0,0,0,0,0],'01-01-2017': [0,0,0,0,0,0], 
                             '01-01-2018': [0,0,0,0,0,0],'01-01-2019': [0,0,0,0,0,0],'01-01-2020': [0,0,0,0,0,0]})

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2013'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2014'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2015'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2016'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2017'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2018'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2019'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2020'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

#df_corr = df_corr.set_index('Labels')
df_dtw_picture = df_dtw.style.background_gradient(cmap='RdYlGn_r',axis=0)
#dfi.export(df_dtw_picture,'../results/'+name_of_df+'.png')
df_dtw_picture

,DTW-Distance to Price,01-01-2012,01-01-2013,01-01-2014,01-01-2015,01-01-2016,01-01-2017,01-01-2018,01-01-2019,01-01-2020
0,RavenPack,2258,1754,1214,885,699,763,687,626,455
1,FinBERT,598,553,468,960,625,347,232,244,182
2,CrudeBERT,635,665,448,468,441,536,224,89,94
3,CrudeBERT_GT,293,237,153,171,136,194,99,47,57
4,CrudeBERT_Plus,438,376,363,254,239,405,133,41,41
5,CrudeBERT_Plus_GT,279,234,202,147,93,197,77,45,40


In [7]:
Price = "Price_change_normalized"
RavenPack = "Daily_Sum_RavenPack_normalized"
FinBERT = "Daily_Sum_FinBERT_normalized"
CrudeBERT = "Daily_Sum_CrudeBERT_normalized"
CrudeBERT_GT = "Daily_Sum_CrudeBERT_GT_normalized"
CrudeBERT_Plus = "Daily_Sum_CrudeBERT_Plus_normalized"
CrudeBERT_Plus_GT = "Daily_Sum_CrudeBERT_Plus_GT_normalized"

selected_data = ['Date', Price, RavenPack, FinBERT, CrudeBERT, CrudeBERT_GT, CrudeBERT_Plus, CrudeBERT_Plus_GT]

dataset_interpolate = dataset_all[selected_data].select_dtypes(exclude='datetime64[ns]').interpolate(method="linear")
dataset_interpolate = dataset_interpolate.dropna()
dataset_interpolate["Date"] = dataset_all["Date"] #Filling in NaN in a Series via linear interpolation

dataset_propagate = dataset_all[selected_data].fillna(method="ffill")
dataset_propagate = dataset_propagate.dropna() #Propagates NaN-values forward or backward

dataset_drop = dataset_all[selected_data].dropna() #NaN-values are dropped

dataset = dataset_propagate

average_price = dataset[Price].mean()
dataset[Price] = dataset.apply(lambda row: row[Price] - average_price, axis=1)

start_date = '01-01-2012'
end_date = '04-01-2021'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

name_of_df = 'DTW-Distance to Price Change'
df_dtw = pd.DataFrame(data={name_of_df: ['RavenPack', 'FinBERT', 'CrudeBERT', 'CrudeBERT_GT', 'CrudeBERT_Plus', 'CrudeBERT_Plus_GT'],
                             '01-01-2012': [0,0,0,0,0,0],'01-01-2013': [0,0,0,0,0,0],'01-01-2014': [0,0,0,0,0,0],
                             '01-01-2015': [0,0,0,0,0,0],'01-01-2016': [0,0,0,0,0,0],'01-01-2017': [0,0,0,0,0,0], 
                             '01-01-2018': [0,0,0,0,0,0],'01-01-2019': [0,0,0,0,0,0],'01-01-2020': [0,0,0,0,0,0]})

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2012'
end_date = '01-01-2013'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2013'
end_date = '01-01-2014'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2014'
end_date = '01-01-2015'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2015'
end_date = '01-01-2016'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2016'
end_date = '01-01-2017'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2017'
end_date = '01-01-2018'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2018'
end_date = '01-01-2019'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2019'
end_date = '01-01-2020'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2020'
end_date = '01-01-2021'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

#df_corr = df_corr.set_index('Labels')
df_dtw_picture = df_dtw.style.background_gradient(cmap='RdYlGn_r',axis=0)
#dfi.export(df_dtw_picture,'../results/'+name_of_df+'.png')
df_dtw_picture

,DTW-Distance to Price Change,01-01-2012,01-01-2013,01-01-2014,01-01-2015,01-01-2016,01-01-2017,01-01-2018,01-01-2019,01-01-2020
0,RavenPack,85,78,63,84,84,63,91,90,84
1,FinBERT,104,105,96,82,90,76,97,98,96
2,CrudeBERT,79,77,66,76,69,66,91,83,84
3,CrudeBERT_GT,75,89,73,98,87,44,60,55,51
4,CrudeBERT_Plus,76,78,72,82,86,70,84,81,80
5,CrudeBERT_Plus_GT,62,83,79,93,89,47,52,52,53


In [8]:
Price = "Price_change_normalized"
RavenPack = "Daily_Sum_RavenPack_normalized"
FinBERT = "Daily_Sum_FinBERT_normalized"
CrudeBERT = "Daily_Sum_CrudeBERT_normalized"
CrudeBERT_GT = "Daily_Sum_CrudeBERT_GT_normalized"
CrudeBERT_Plus = "Daily_Sum_CrudeBERT_Plus_normalized"
CrudeBERT_Plus_GT = "Daily_Sum_CrudeBERT_Plus_GT_normalized"
selected_data = ['Date', Price, RavenPack, FinBERT, CrudeBERT, CrudeBERT_GT, CrudeBERT_Plus, CrudeBERT_Plus_GT]


dataset_interpolate = dataset_all[selected_data].select_dtypes(exclude='datetime64[ns]').interpolate(method="linear")
dataset_interpolate = dataset_interpolate.dropna()
dataset_interpolate["Date"] = dataset_all["Date"] #Filling in NaN in a Series via linear interpolation

dataset_propagate = dataset_all[selected_data].fillna(method="ffill")
dataset_propagate = dataset_propagate.dropna() #Propagates NaN-values forward or backward

dataset_drop = dataset_all[selected_data].dropna() #NaN-values are dropped

dataset = dataset_interpolate

average_price = dataset[Price].mean()
dataset[Price] = dataset.apply(lambda row: row[Price] - average_price, axis=1)

start_date = '01-01-2012'
end_date = '04-01-2021'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

name_of_df = 'DTW-Distance to Price Change'
df_dtw = pd.DataFrame(data={name_of_df: ['RavenPack', 'FinBERT', 'CrudeBERT', 'CrudeBERT_GT', 'CrudeBERT_Plus', 'CrudeBERT_Plus_GT'],
                             '01-01-2012': [0,0,0,0,0,0],'01-01-2013': [0,0,0,0,0,0],'01-01-2014': [0,0,0,0,0,0],
                             '01-01-2015': [0,0,0,0,0,0],'01-01-2016': [0,0,0,0,0,0],'01-01-2017': [0,0,0,0,0,0], 
                             '01-01-2018': [0,0,0,0,0,0],'01-01-2019': [0,0,0,0,0,0],'01-01-2020': [0,0,0,0,0,0]})

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2013'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2014'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2015'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2016'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2017'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2018'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2019'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

start_date = '01-01-2020'
dataset_date = dataset.copy()
selected_date = (dataset_date['Date'] >= start_date) & (dataset_date['Date'] <= end_date)
dataset_date = dataset_date.loc[selected_date]
dataset_dtw = dataset_date

Price_dtw = dataset_dtw[Price]
RavenPack_dtw = dataset_dtw[RavenPack]
FinBERT_dtw = dataset_dtw[FinBERT]
CrudeBERT_dtw = dataset_dtw[CrudeBERT]
CrudeBERT_GT_dtw = dataset_dtw[CrudeBERT_GT]
CrudeBERT_Plus_dtw = dataset_dtw[CrudeBERT_Plus]
CrudeBERT_Plus_GT_dtw = dataset_dtw[CrudeBERT_Plus_GT]
dtw_distance_RP, warp_path = fastdtw(Price_dtw, RavenPack_dtw, dist=euclidean)
dtw_distance_FB, warp_path = fastdtw(Price_dtw, FinBERT_dtw, dist=euclidean)
dtw_distance_CB, warp_path = fastdtw(Price_dtw, CrudeBERT_dtw, dist=euclidean)
dtw_distance_CB_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_GT_dtw, dist=euclidean)
dtw_distance_CB_Plus, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_dtw, dist=euclidean)
dtw_distance_CB_Plus_GT, warp_path = fastdtw(Price_dtw, CrudeBERT_Plus_GT_dtw, dist=euclidean)
df_dtw.loc[[0],str(start_date)] = round(dtw_distance_RP, 0)
df_dtw.loc[[1],str(start_date)] = round(dtw_distance_FB, 0)
df_dtw.loc[[2],str(start_date)] = round(dtw_distance_CB, 0)
df_dtw.loc[[3],str(start_date)] = round(dtw_distance_CB_GT, 0)
df_dtw.loc[[4],str(start_date)] = round(dtw_distance_CB_Plus, 0)
df_dtw.loc[[5],str(start_date)] = round(dtw_distance_CB_Plus_GT, 0)

#df_corr = df_corr.set_index('Labels')
df_dtw_picture = df_dtw.style.background_gradient(cmap='RdYlGn_r',axis=0)
#dfi.export(df_dtw_picture,'../results/'+name_of_df+'.png')
df_dtw_picture

,DTW-Distance to Price Change,01-01-2012,01-01-2013,01-01-2014,01-01-2015,01-01-2016,01-01-2017,01-01-2018,01-01-2019,01-01-2020
0,RavenPack,652,573,507,456,387,307,247,173,92
1,FinBERT,862,745,630,565,465,376,313,192,115
2,CrudeBERT,627,602,496,455,377,312,256,166,90
3,CrudeBERT_GT,609,510,465,369,294,208,164,108,63
4,CrudeBERT_Plus,645,596,492,448,357,307,236,160,85
5,CrudeBERT_Plus_GT,553,506,426,369,282,193,153,103,66
